# 用线性回归模型或者lightGBM预测前10天的新增

In [6]:
#
import os
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import lightgbm as lgb
#import xgboost as xgb
from lightgbm import LGBMRegressor 
import time
import warnings
from math import sqrt
from matplotlib import pyplot
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge
warnings.filterwarnings("ignore")

In [7]:
#训练数据给的是5.1到6.29日
train_dataA=pd.read_csv('dataset/features/features_A.csv')
train_dataB=pd.read_csv('dataset/features/features_B.csv')
train_dataC=pd.read_csv('dataset/features/features_C.csv')
train_dataD=pd.read_csv('dataset/features/features_D.csv')
train_dataE=pd.read_csv('dataset/features/features_E.csv')
train_dataF=pd.read_csv('dataset/features/features_F.csv')
train_dataG=pd.read_csv('dataset/features/features_G.csv')
train_dataH=pd.read_csv('dataset/features/features_H.csv')
train_dataI=pd.read_csv('dataset/features/features_I.csv')
train_dataJ=pd.read_csv('dataset/features/features_J.csv')
train_dataK=pd.read_csv('dataset/features/features_K.csv')
train_dic={'A':train_dataA,'B':train_dataB,'C':train_dataC,'D': train_dataD,'E':train_dataE,
        'F':train_dataF, 'G':train_dataG,'H':train_dataH,'I': train_dataI,'J':train_dataJ,'K':train_dataK}
def clear_data(df):
    return df.drop(['humidity','wind_force','wind_direction',
                    'wind_speed','weather_type','temperature','ext_transfer','iner_transfer','density','migration'],axis=1)
train_data={}
for city_id in ['A','B','C','D','E','F','G','H','I','J','K']:
    train_data[city_id]=clear_data(train_dic[city_id])

In [14]:
model_lgbm = LGBMRegressor(n_estimators=10, metric='mae', random_state=2020, min_child_samples=5, min_child_weight=0.000001)

In [1]:
def reserve_target(data,index):
    max_id=data.index(max(data))
    if max_id==len(data)-1:#最后一个点为最大值
        return list(index)
    else:
        return list(index[max_id:])
#保留最大值往后的值
for city_id in ['A','B','C','D','E','F','G','H','I','J','K']:
    grouped=train_data[city_id].groupby('region_id')
    reserve_list=[]
    for ciy_id,group in grouped:
        reserve_list+=reserve_target(list(group['index'].values),group.index)
    train_data[city_id]=train_data[city_id].loc[reserve_list].reset_index(drop=True)
def calculate_lag(df, lag_list, column):
    for lag in lag_list:
        column_lag = column + "_" + str(lag)
        df[column_lag] = df[column].shift(lag, fill_value=0)
    return df

def calculate_trend(df, lag_list, column):
    for lag in lag_list:
        trend_column_lag = "Trend_" + column + "_" + str(lag)
        df[trend_column_lag] = (df[column].shift(0, fill_value=0) - 
                                df[column].shift(lag, fill_value=0))/df[column].shift(lag, fill_value=0.001)
    return df

def get_feature(df):
    df = calculate_lag(df, range(1,7), 'index')
    #all_data = calculate_lag(all_data, range(1,7), 'migration')
    df = calculate_trend(df, range(1,7), 'index')
    #all_data = calculate_trend(all_data, range(1,7), 'migration')
    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.fillna(0, inplace=True)
    return df
def get_test(train_df):
    test_data=train_df[-1:].reset_index(drop=True)
    last_raw=train_df[-1:].reset_index(drop=True)
    pres1=['index_'+str(i) for i in range(1,7)]
    pres2=['Trend_index_'+str(i) for i in range(1,7)]
    test_data.loc[0,'index_1']=last_raw.loc[0,'index']
    for i in range(len(pres1)-1):
        test_data.loc[0,pres1[i+1]]=last_raw.loc[0,pres1[i]]
    test_data.loc[0,'Trend_index_1']=1
    for i in range(len(pres2)-1):
        test_data.loc[0,pres2[i+1]]=last_raw.loc[0,pres2[i]]
    X_test=test_data.drop(['index'],axis=1)
    return X_test
pre_lens=5
pre_dicSeir={}
for city_id in ['A','B','C','D','E','F','G','H','I','J','K']:
    grouped=train_data[city_id].groupby('region_id')
    reserve_list=[]
    pre_dic={}
    for region_id,group in grouped:
        group=group[-20:].reset_index(drop=True)
        group=group.drop(['date','city','region_id'],axis=1)
        print(len(group))
        pyplot.plot(group.values,'green')
        pres=[]
        for i in range(pre_lens):
            #print(len(group))
            train_df=get_feature(group)
            X_train=train_df.drop(['index'],axis=1)
            Y_train=train_df['index']
            X_test=get_test(train_df)
            regr = linear_model.LinearRegression()#线性回归模型，可选树模型
            model_choose=regr
            model_choose.fit(X_train, Y_train)
            y_pred = model_choose.predict(X_test)
            raw={'index':y_pred[0]}
            pres.append(y_pred[0])
            group=group.append(raw,ignore_index=True)
        print(region_id,pres)
        pre_dic.update({region_id:pres})
        pyplot.plot([i for i in range(len(group)-len(pres),len(group))],pres, color='red')
        save_dir='AR_fit_city_lgb/'+city_id
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(os.path.join(save_dir,str(region_id)+'.png'))
        pyplot.show()
        pyplot.close()
    pre_dicSeir.update({city_id:pre_dic})

In [2]:
import pandas as pd
re_sub=pd.read_csv('submits/submission1_16.csv',header=None)
re_sub.columns = ['city','region_id','date', 'index']
import random
tmp_list=[]
ex={'A':[],'C':[],'I':[8],'J':[3],'K':[0]}
def convert_date(date):
    if date==21200630:
        return 0
    else:
        return date-21200700
for num,city,region_id,date,value in re_sub.itertuples():
    if date<21200702:
        if city in ['A','C','I','J','K']:
            replace_lgb=pre_dicSeir[city][region_id][convert_date(date)]
            if replace_lgb<0:
                replace_lgb=0
            if region_id not in ex[city]:
                re_sub.loc[num,'index']=int(np.ceil(replace_lgb))
            #re_sub.loc[num,'index']=int(np.ceil(replace_lgb))
re_sub.to_csv('submits/submission7_23v1.csv',index=False,header=None)
re_sub.head(2)